# GenerativeAI4DS-I
## Lab. Math Tutor Assistant


##  What I hope you'll get out of this lab
* The feeling that you'll "know where to start" when you have to consume OpenAI services.
* Follow OpenAI's best practices on how to develop assistants

In [1]:
!pip install openai

In [2]:
from openai import OpenAI
import os
import json
from IPython.core.display import display, HTML

In [3]:
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

# 1. You have to get your [OpenAI API Key](https://platform.openai.com/account/api-keys)

In [4]:
# Used by the agent in this tutorial
os.environ["OPENAI_API_KEY"] = "YOU-NEED-YOUR-OWN-KEY"

In [5]:
client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
)

# 2. We consume an existing Math Tutor Assistant
An Assistant represents an entity that can be configured to respond to a user's messages using several parameters like model, instructions, and tools.





### 2.1. We instantiate an existing assistant

In [6]:
assistant = client.beta.assistants.retrieve("asst_l2lH2H37uZKZHIDcMUY92EwI")
show_json(assistant)

{'id': 'asst_l2lH2H37uZKZHIDcMUY92EwI',
 'created_at': 1706000179,
 'description': None,
 'instructions': 'You are a personal math tutor. Answer questions to students in high school',
 'metadata': {},
 'model': 'gpt-3.5-turbo-16k',
 'name': 'Math Tutor (High School)',
 'object': 'assistant',
 'tools': [],
 'response_format': 'auto',
 'temperature': 1.0,
 'tool_resources': {'code_interpreter': None, 'file_search': None},
 'top_p': 1.0}

### 2.2. We create a thread

A Thread represents a conversation between a user and one or many Assistants. You can create a Thread when a user (or your AI application) starts a conversation with your Assistant.

In [7]:
thread = client.beta.threads.create()

In [13]:
thread

Thread(id='thread_tCiCFi1B1WWrzYfYg33jbDTm', created_at=1716819424, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

### 2.3 We add a message to the thread

The contents of the messages your users or applications create are added as Message objects to the Thread. Messages can contain both text and files. There is no limit to the number of Messages you can add to Threads — we smartly truncate any context that does not fit into the model's context window.

In [8]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

show_json(message)

{'id': 'msg_xhzJ7Atu38lngaUM0gaQDqr6',
 'assistant_id': None,
 'attachments': [],
 'completed_at': None,
 'content': [{'text': {'annotations': [],
    'value': 'I need to solve the equation `3x + 11 = 14`. Can you help me?'},
   'type': 'text'}],
 'created_at': 1716819424,
 'incomplete_at': None,
 'incomplete_details': None,
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'status': None,
 'thread_id': 'thread_tCiCFi1B1WWrzYfYg33jbDTm'}

### 2.4. We create a run

Once all the user Messages have been added to the Thread, you can Run the Thread with any Assistant. Creating a Run uses the model and tools associated with the Assistant to generate a response. These responses are added to the Thread as assistant Messages.

In [9]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)
show_json(run)

{'id': 'run_zr0eOfg6a5PkTPGs7tt8k0xD',
 'assistant_id': 'asst_l2lH2H37uZKZHIDcMUY92EwI',
 'cancelled_at': None,
 'completed_at': 1716819429,
 'created_at': 1716819425,
 'expires_at': None,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': 'Please address the user as Jane Doe. The user has a premium account.',
 'last_error': None,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-16k',
 'object': 'thread.run',
 'required_action': None,
 'response_format': 'auto',
 'started_at': 1716819425,
 'status': 'completed',
 'thread_id': 'thread_tCiCFi1B1WWrzYfYg33jbDTm',
 'tool_choice': 'auto',
 'tools': [],
 'truncation_strategy': {'type': 'auto', 'last_messages': None},
 'usage': {'completion_tokens': 164, 'prompt_tokens': 46, 'total_tokens': 210},
 'temperature': 1.0,
 'top_p': 1.0,
 'tool_resources': {}}

### 2.5 We observe the response
Runs are asynchronous, which means you'll want to monitor their status by polling the Run object until a terminal status is reached. For convenience, the 'create and poll' SDK helpers assist both in creating the run and then polling for its completion.

In [10]:
if run.status == 'completed':
  responses = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(responses)
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_tUWsw2gwbXa3JlTTiGW2QDwI', assistant_id='asst_l2lH2H37uZKZHIDcMUY92EwI', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="Of course, Jane Doe! I'd be happy to help you solve the equation `3x + 11 = 14`. The goal here is to isolate the variable `x` on one side of the equation. Let's go step by step:\n\n1. Subtract 11 from both sides of the equation:\n   `3x + 11 - 11 = 14 - 11`\n   Simplifying, we get:\n   `3x = 3`\n\n2. Divide both sides of the equation by 3 to solve for `x`:\n   `3x/3 = 3/3`\n   This simplifies to:\n   `x = 1`\n\nSo, the solution to the equation `3x + 11 = 14` is `x = 1`."), type='text')], created_at=1716819426, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_zr0eOfg6a5PkTPGs7tt8k0xD', status=None, thread_id='thread_tCiCFi1B1WWrzYfYg33jbDTm'), Message(id='msg_xhzJ7Atu38lngaUM0gaQDqr6', assistant_id=None, attachment

In [11]:
show_json(responses.data[0].content[0])

{'text': {'annotations': [],
  'value': "Of course, Jane Doe! I'd be happy to help you solve the equation `3x + 11 = 14`. The goal here is to isolate the variable `x` on one side of the equation. Let's go step by step:\n\n1. Subtract 11 from both sides of the equation:\n   `3x + 11 - 11 = 14 - 11`\n   Simplifying, we get:\n   `3x = 3`\n\n2. Divide both sides of the equation by 3 to solve for `x`:\n   `3x/3 = 3/3`\n   This simplifies to:\n   `x = 1`\n\nSo, the solution to the equation `3x + 11 = 14` is `x = 1`."},
 'type': 'text'}

In [12]:
display(HTML(responses.data[0].content[0].text.value))

### 2.6 We add more messages to the same thread as needed

In [14]:
message2 = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="I need to solve the equation `7x^2-3=0`. Can you help me?"
)

show_json(message2)

{'id': 'msg_Wos0xoS4gUdvibXN55CEGV54',
 'assistant_id': None,
 'attachments': [],
 'completed_at': None,
 'content': [{'text': {'annotations': [],
    'value': 'I need to solve the equation `7x^2-3=0`. Can you help me?'},
   'type': 'text'}],
 'created_at': 1716819734,
 'incomplete_at': None,
 'incomplete_details': None,
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'status': None,
 'thread_id': 'thread_tCiCFi1B1WWrzYfYg33jbDTm'}

In [15]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)
show_json(run)

{'id': 'run_OfuOZKubzx8Ajopx6C8pQiHU',
 'assistant_id': 'asst_l2lH2H37uZKZHIDcMUY92EwI',
 'cancelled_at': None,
 'completed_at': 1716819792,
 'created_at': 1716819787,
 'expires_at': None,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': 'Please address the user as Jane Doe. The user has a premium account.',
 'last_error': None,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-16k',
 'object': 'thread.run',
 'required_action': None,
 'response_format': 'auto',
 'started_at': 1716819787,
 'status': 'completed',
 'thread_id': 'thread_tCiCFi1B1WWrzYfYg33jbDTm',
 'tool_choice': 'auto',
 'tools': [],
 'truncation_strategy': {'type': 'auto', 'last_messages': None},
 'usage': {'completion_tokens': 343,
  'prompt_tokens': 237,
  'total_tokens': 580},
 'temperature': 1.0,
 'top_p': 1.0,
 'tool_resources': {}}

In [16]:
if run.status == 'completed':
  responses = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(responses)
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_3HWhE5j5r6Z8kSaPGLBlERmX', assistant_id='asst_l2lH2H37uZKZHIDcMUY92EwI', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="Absolutely, Jane Doe! I can assist you in solving the equation `7x^2 - 3 = 0`. This is a quadratic equation, which can be solved using various methods, such as factoring, completing the square, or using the quadratic formula.\n\nIn this case, we will use the quadratic formula to find the value(s) of `x`. The quadratic formula states that for an equation in the form `ax^2 + bx + c = 0`, the solutions for `x` are given by:\n\n`x = (-b ± √(b^2 - 4ac)) / (2a)`\n\nNow let's apply the quadratic formula to our equation `7x^2 - 3 = 0`:\n\nIn this case:\na = 7\nb = 0 (no x-term)\nc = -3\n\nPlugging these values into the quadratic formula, we get:\n\n`x = (-0 ± √(0^2 - 4 * 7 * -3)) / (2 * 7)`\n\nSimplifying further:\n\n`x = ±√(0 + 84) / 14`\n\n`x = ±√(84) / 14`\n\n`x = ±√(4 * 21) / 14`\

In [17]:
display(HTML(responses.data[0].content[0].text.value))